# Final Project : Classification(Premium Default Prediction)

# PROBLEM DEFINITION

# HPOTHESIS GENERATION

# DATA EXTRACTION AND COLLECTION

# Importing Libraries

In [ ]:
# importing libraries
# For mathematical calculations and array manipulation
import pandas as pd
import numpy as np                   

import seaborn as sns             # For data visualization
import matplotlib.pyplot as plt   # For plotting graphs
%matplotlib inline

import os
import warnings                   # To ignore any warnings
warnings.filterwarnings("ignore")


from sklearn.preprocessing import StandardScaler, MinMaxScaler   #standard scaler

# Evaluation metrics
from sklearn.metrics import confusion_matrix  #evaluation metrics
from sklearn.metrics import classification_report

#MODEL BUILDING
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron 
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

#parameter improving
from sklearn.model_selection import GridSearchCV

# READING DATA

In [ ]:
# loading the data
train = pd.read_csv('train credit.csv')
test = pd.read_csv('test credit.csv')
combine = [train, test]
#combine used to ensure changes done to complete dataset

Let's check the features present in our data and then we will look at their data types.

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.shape, test.shape

In [ ]:
#printing first five rows of the dataset
train.head()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
test.tail()

# VARIABLE IDENTIFICATION

In [ ]:
# Print data types for each variable
train.dtypes

In [ ]:
test.dtypes

# We can see there are two format of data types:

1. **object**: Object format means variables are categorical. Categorical variables in our dataset are: job, marital, education, default, housing, loan, contact, month, poutcome, subscribed

2. **int64**: It represents the integer variables. Integer variables in our dataset are: ID, age, balance, day, duration, campaign, pdays, previous

# UNIVARIATE ANALYSIS

In [ ]:
train.describe()

93% of the people paid their premium.

In [ ]:
test.describe()

In [ ]:
train.describe(percentiles = [.08, .07, .06])

In [ ]:
test.describe(percentiles = [.08, .07, .06])

# BIVARIATE ANALYSIS

In [ ]:
plt.figure(figsize = (15, 6))
sns.heatmap(train.corr(), annot = True)
plt.show()

# DATA EXPLORATION AND TRANSFORMATION

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull()

In [ ]:
test.isnull()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# CONVERTING AGE IN DAYS TO YEARS AND DROPPING PREVIOUS ONE 
for dataset in combine: 
    dataset['age'] = dataset['age_in_days']//365
    dataset.drop(['age_in_days'], axis = 1, inplace = True)
train.head()

In [ ]:
#GROUPING SOURCING CHANNEL
train[['sourcing_channel', 'target']].groupby('sourcing_channel', as_index = False).mean()

We might need to make income groups to understand the relations better

In [ ]:
#BINNING INCOME OF THE TRAIN DATASET AND GRPUPING ON THE BASIS OF TARGET
train['Incomeclass'] = pd.cut(train['Income'], 5)
train[['Incomeclass', 'target']].groupby('Incomeclass', as_index = False).count()

In [ ]:
#Let's standardize our data by using a standard scaler
scaler = MinMaxScaler()
scaler = scaler.fit(train[['Income']])
xscale = scaler.transform(train[['Income']])
xscale

In [ ]:
#scaler.mean
print(scaler.scale_)

In [ ]:
#ADDING SCALED INCOME
train['scale_income'] = xscale
train.head()

In [ ]:
#BINNING INCOMECLASS AS WELL OF THE TRAIN DATASET AND GRPUPING ON THE BASIS OF TARGET
train['Incomeclass'] = pd.cut(train['scale_income'], 5)
train[['Incomeclass', 'target']].groupby('Incomeclass', as_index = False).count()

# Outlier Treatment For Income

In [ ]:
plt.hist(train['Income'])
plt.show()

In [ ]:
upper_bound = 0.95
lower_bound = 0.1
result = train['Income'].quantile([lower_bound, upper_bound])
print(result)

So, we can collect all the values in this range and let go of the other ones.

In [ ]:
true_index = (train['Income'] < result.loc[upper_bound])
true_index

In [ ]:
false_index = ~true_index

In [ ]:
no_outlier_train = train[true_index].copy()
no_outlier_train.head()

In [ ]:
no_outlier_train.describe()
#removed some outliers from every features availaible

In [ ]:
#Let's make groups for the new income range

In [ ]:
#BINNING INCOMECLASS AS WELL OF THE TRAIN DATASET AND GRPUPING ON THE BASIS OF TARGET
no_outlier_train['Incomeclass'] = pd.cut(no_outlier_train['Income'], 5)
no_outlier_train[['Incomeclass', 'target']].groupby('Incomeclass', as_index = False).count()

In [ ]:
#dividing income into a proper classes
combine = [train, test]
for dataset in combine: 
    dataset.loc[ dataset['Income'] <= 23603.99, 'Income'] = 0
    dataset.loc[(dataset['Income'] > 23603.99) & (dataset['Income'] <= 109232.0), 'Income'] = 1
    dataset.loc[(dataset['Income'] > 109232.0) & (dataset['Income'] <= 194434.0), 'Income'] = 2
    dataset.loc[(dataset['Income'] > 194434.0) & (dataset['Income'] <= 279636.0), 'Income'] = 3
    dataset.loc[(dataset['Income'] > 279636.0) & (dataset['Income'] <= 364838.0), 'Income'] = 4
    dataset.loc[(dataset['Income'] > 364838.0) & (dataset['Income'] <= 450040.0), 'Income'] = 5
    dataset.loc[ dataset['Income'] > 450040.0, 'Income'] = 6
    
train.head()

In [ ]:
train.loc[false_index, 'Income'] = 5
train.head()

In [ ]:
train.drop(['Incomeclass', 'scale_income'], axis = 1, inplace = True)
train.head()

making classes for Age

In [ ]:
train['Ageclass'] = pd.cut(train['age'], 5)
train[['Ageclass', 'target']].groupby('Ageclass', as_index = False).count()

The Age feature lies between 20 to 103.

In [ ]:
#assigning numbers to the ageclass
for dataset in combine:    
    dataset.loc[ dataset['age'] <= 37.4, 'age'] = 0
    dataset.loc[(dataset['age'] > 37.4) & (dataset['age'] <= 53.8), 'age'] = 1
    dataset.loc[(dataset['age'] > 53.8) & (dataset['age'] <= 70.2), 'age'] = 2
    dataset.loc[(dataset['age'] > 70.2) & (dataset['age'] <= 86.6), 'age'] = 3
    dataset.loc[ dataset['age'] > 86.6, 'age'] = 4
combine = [train, test]
train.head()

In [ ]:
train.drop('Ageclass', axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
#grouping of mean und_score based on the ageclass
train[['age', 'application_underwriting_score']].groupby('age').mean()

In [ ]:
#binning the number of premium paid
train['Premiumclass'] = pd.cut(train['no_of_premiums_paid'], 5)

In [ ]:
train.head()

In [ ]:
#grouping premiums based on the application_underwriting _scores
train[['Premiumclass', 'application_underwriting_score']].groupby('Premiumclass').count()

In [ ]:
print(train['application_underwriting_score'].mean())
print(train['application_underwriting_score'].std())

In [ ]:
print(train[train['sourcing_channel'] == 'A']['application_underwriting_score'].std())

In [ ]:
#grouping the sourcing channel
train[['sourcing_channel', 'target']].groupby('sourcing_channel', as_index = False).mean()

In [ ]:
print(train[train['sourcing_channel'] == 'C']['application_underwriting_score'].std())

In [ ]:
train[['sourcing_channel', 'application_underwriting_score']].groupby('sourcing_channel', as_index = False).mean()

In [ ]:
train[['residence_area_type', 'application_underwriting_score']].groupby('residence_area_type', as_index = False).mean()

setting the value of underwriting score based on the sourcing channel. 

In [ ]:
train.dtypes

In [ ]:
combine = [train, test]
for dataset in combine: 
    mask1 = dataset['application_underwriting_score'].isnull()
    for source in ['A', 'B', 'C', 'D', 'E']:
        mask2 = (dataset['sourcing_channel'] == source)
        source_mean = dataset[dataset['sourcing_channel'] == source]['application_underwriting_score'].mean()
        dataset.loc[mask1 & mask2, 'application_underwriting_score'] = source_mean

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
test[test['Count_3-6_months_late'].isnull()]

In [ ]:
#visualizing late premiums to treat missing value and outliers
sns.countplot(x = 'Count_3-6_months_late', data = train, hue = 'target')

In [ ]:
sns.countplot(x = 'Count_6-12_months_late', data = train, hue = 'target')

In [ ]:
sns.countplot(x = 'Count_more_than_12_months_late', data = train, hue = 'target')

In [ ]:
#intoducing late_premium for all the late premiums
combine = [train, test]
for dataset in combine: 
    dataset['late_premium'] = 0.0


In [ ]:
train.head()

In [ ]:
#finding late premiums and adding]
combine = [train, test]
for dataset in combine:
        dataset.loc[(dataset['Count_3-6_months_late'].isnull()),  'late_premium'] = np.NaN
        dataset.loc[(dataset['Count_3-6_months_late'].notnull()), 'late_premium'] = dataset['Count_3-6_months_late'] + dataset['Count_6-12_months_late'] + dataset['Count_more_than_12_months_late']
        

In [ ]:
train.head()

In [ ]:
#correlation between every feature of train
plt.figure(figsize = (15, 6))
sns.heatmap(train.corr(), annot = True)

In [ ]:
#correlation between every feature of test
plt.figure(figsize = (15, 6))
sns.heatmap(test.corr(), annot = True)

In [ ]:
sns.countplot(x = 'late_premium', data = train, hue = 'target')

In [ ]:
#assuming that max late premiums are 7 and min is 2 

In [ ]:
train[['late_premium', 'target']].groupby('late_premium').mean()

In [ ]:
#if late premium is null and target is 0 then loan not granted and max is 7
#if late premium is null and target is 1 then loan granted and min is 2
#if 
train.loc[(train['target'] == 0) & (train['late_premium'].isnull()),'late_premium'] = 7
train.loc[(train['target'] == 1) & (train['late_premium'].isnull()),'late_premium'] = 2

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#missing value treatment in test dataset due unknown target variable by putting median
prem = np.zeros(5)
for dataset in [test]:
    for i in range(1, 6):
        df = dataset[(dataset['Income'] == i)]['late_premium'].dropna()
        premium_guess = df.median()
        prem[i - 1] = int(premium_guess) 

    for j in range(1, 6):
        dataset.loc[(dataset.late_premium.isnull()) & (dataset.Income == j), 'late_premium'] = prem[j - 1] + 1

    dataset['late_premium'] = dataset['late_premium'].astype(int)

In [ ]:
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
train.drop(['Count_3-6_months_late', 'Count_6-12_months_late', 'Count_more_than_12_months_late'], axis = 1, inplace = True)

In [ ]:
test.drop(['Count_3-6_months_late', 'Count_6-12_months_late', 'Count_more_than_12_months_late'], axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# tranforming Area Type and sourcing channel to numerical values using dictionary
combine = [train, test]
for dataset in combine: 
    dataset['residence_area_type'] = dataset['residence_area_type'].map( {'Urban' : 1, 'Rural' : 0} )
    dataset['sourcing_channel'] = dataset['sourcing_channel'].map( {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3, 'E' : 4} )

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['application_underwriting_score'] = train['application_underwriting_score']/100

In [ ]:
train.head()

In [ ]:
#treating number of premiums
result = train['no_of_premiums_paid'].quantile([.95])
print(result)

In [ ]:
true_index = train['no_of_premiums_paid'] < result.loc[.95]
false_index = ~true_index
true_index

In [ ]:
train['Premiumclass'] = pd.cut(train[true_index]['no_of_premiums_paid'], 4)
train[['Premiumclass', 'application_underwriting_score']].groupby('Premiumclass').count()

In [ ]:
# combine = [train_data, test_data]
# for dataset in combine: 
#     dataset.loc[ dataset['no_of_premiums_paid'] <= 6.25, 'no_of_premiums_paid'] = 0
#     dataset.loc[(dataset['no_of_premiums_paid'] > 6.25) & (dataset['no_of_premiums_paid'] <= 10.5), 'no_of_premiums_paid'] = 1
#     dataset.loc[(dataset['no_of_premiums_paid'] > 10.50) & (dataset['no_of_premiums_paid'] <= 14.75), 'no_of_premiums_paid'] = 2
#     dataset.loc[(dataset['no_of_premiums_paid'] > 14.75) & (dataset['no_of_premiums_paid'] <= 19.0), 'no_of_premiums_paid'] = 3
#     dataset.loc[ dataset['no_of_premiums_paid'] > 19.0, 'no_of_premiums_paid'] = 4
    
# train_data.drop('PremBand', axis = 1, inplace = True)
# train_data.head()

In [ ]:
#converting percentage premiums
train['Premiumclass'] = pd.cut(train['perc_premium_paid_by_cash_credit'], 4)
train[['Premiumclass', 'target']].groupby('Premiumclass').mean()

In [ ]:
combine = [train, test]
for dataset in combine: 
    dataset.loc[ dataset['perc_premium_paid_by_cash_credit'] <= 0.25, 'perc_premium_paid_by_cash_credit'] = 0
    dataset.loc[(dataset['perc_premium_paid_by_cash_credit'] > 0.25) & (dataset['perc_premium_paid_by_cash_credit'] <= 0.5), 'perc_premium_paid_by_cash_credit'] = 1
    dataset.loc[(dataset['perc_premium_paid_by_cash_credit'] > 0.5) & (dataset['perc_premium_paid_by_cash_credit'] <= 0.75), 'perc_premium_paid_by_cash_credit'] = 2
    dataset.loc[ dataset['perc_premium_paid_by_cash_credit'] > 0.75, 'perc_premium_paid_by_cash_credit'] = 3
train.drop('Premiumclass', axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train[['perc_premium_paid_by_cash_credit', 'late_premium']] = train[['perc_premium_paid_by_cash_credit', 'late_premium']].astype(int)
test[['perc_premium_paid_by_cash_credit']] = test[['perc_premium_paid_by_cash_credit']].astype(int)

DataFrame.astype() method is used to cast a pandas object to a specified dtype. astype() function also provides the capability to convert any suitable existing column to categorical type.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

# MODEL BUILDING

In [ ]:
X_train = train.drop(['id', 'target','perc_premium_paid_by_cash_credit'], axis = 1).copy()
y_train = train['target']
X_test = test.drop(['id', 'perc_premium_paid_by_cash_credit'], axis = 1).copy()
print(X_train.shape, y_train.shape, X_test.shape)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

# logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
print("y_pred")
print(y_pred)

score_logreg = round(logreg.score(X_train, y_train) * 100, 2)
print("Score")
print(score_logreg)

conf_mat = confusion_matrix(logreg.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(logreg.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

In [ ]:
coeff = pd.DataFrame(train.columns.delete(0))
coeff.columns = ['Feature']
coeff['Correlation'] = pd.Series(logreg.coef_[0])
coeff.sort_values(by = 'Correlation', ascending = False)

# Gaussian Naive Bayes

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)

y_pred = gaussian.predict(X_test)
print("y_pred")
print(y_pred)

score_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
print("Score")
print(score_gaussian)

conf_mat = confusion_matrix(gaussian.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(gaussian.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

# K-Nearest Neighbor

In [ ]:
# create new a knn model
knn2 = KNeighborsClassifier()
# create a dict of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(2, 10)}
# use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)
#fit model to data
knn_gscv.fit(X_train, y_train)

In [ ]:
knn_gscv.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print("y_pred")
print(y_pred)

score_knn = round(knn.score(X_train, y_train) * 100, 2)
print("Score")
print(score_knn)

conf_mat = confusion_matrix(knn.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(knn.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

# Perceptron Algorithm

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

y_pred = perceptron.predict(X_test)
print("y_pred")
print(y_pred)

score_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
print("Score")
print(score_perceptron)

conf_mat = confusion_matrix(perceptron.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(perceptron.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

# Stochastic Gradient Descent

In [ ]:
sto = SGDClassifier()
sto.fit(X_train, y_train)

Y_pred = sto.predict(X_test)
print("y_pred")
print(y_pred)

score_sto = round(sto.score(X_train, y_train) * 100, 2)
print("Score")
print(score_sto)

conf_mat = confusion_matrix(sto.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(sto.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

# Decision Tree

In [ ]:
param_grid = {
    'max_depth': np.arange(2,100)}
#param_grid = {'max_depth': [5, 15, 25],'min_samples_leaf': [1, 3],'max_leaf_nodes': [10, 20, 35, 50]}
dt = DecisionTreeClassifier()
gs = GridSearchCV(dt, param_grid, scoring='f1', cv=5)
gs.fit(X_train, y_train)
print("best params:", gs.best_params_)

In [ ]:
dec = DecisionTreeClassifier(max_depth = 7)
dec.fit(X_train, y_train)

y_pred = dec.predict(X_test)
print("y_pred")
print(y_pred)

score_dec = round(dec.score(X_train, y_train) * 100, 2)
print("Score")
print(score_dec)

conf_mat = confusion_matrix(dec.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(dec.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

# Random Forest

In [ ]:
param_grid = {
    'n_estimators': np.arange(1,10)
}

rf = RandomForestClassifier(random_state=123)
gs = GridSearchCV(rf, param_grid, scoring='f1', cv=5)
gs.fit(X_train, y_train)
print("best params:", gs.best_params_)

In [ ]:
ran = RandomForestClassifier(n_estimators = 10)
ran.fit(X_train, y_train)

y_pred = ran.predict(X_test)
ran.score(X_train, y_train)
print("y_pred")
print(y_pred)

score_ran = round(ran.score(X_train, y_train) * 100, 2)
print("Score")
print(score_ran)

conf_mat = confusion_matrix(ran.predict(X_train),y_train)
print("Confusion Matrix")
print(conf_mat)

print("Classification Report")
print(classification_report(ran.predict(X_train),y_train))

tnr = np.round(conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) * 100,3)
tpr = np.round(conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]) * 100,3)
fpr = np.round(conf_mat[1][0] / (conf_mat[1][0] + conf_mat[0][0]) * 100,3)
print('TPR = ',tpr,'%')
print('TNR = ',tnr,'%')
print('FPR = ',fpr,'%')

# Hyperparameter KNN
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
p = list(range(1,100,3))
parameters = {'n_neighbors':p}
clf = GridSearchCV(knn,param_grid = parameters, scoring = 'roc_auc', cv=10, return_train_score = True)
clf.fit(X_train_res,y_train_res)

train_auc_error = [1 - x for x in clf.cv_reslts_['mean_train_score']]
train_auc_std = np.std(train_auc_error)

test_auc_error = [1 - x for x in clf.cv_results_['mean_test_score']]
test_auc_std = np.std(test_auc_error)

plt.plot(p,train_auc_error,label = 'Train AUC',color = 'orange')
plt.gca().fill_between(parameters,train_auc_error-train_auc_std,train_auc_error+train_auc_std,color= 'orange')

plt.plot(p,test_auc_error,label = 'Test AUC', color = 'darkblue')
plt.gca().fill_between(parameters,test_auc_error-test_auc_std,test_auc_std+test_auc_std,color= 'darkbue')

plt.xlabel('K: Hyperparameter')
plt.ylabel('Errors')
plt.legend(loc = 'lower right')
plt.show()

optimal_k = clf.best_params_.get('n_neighbors')

print('optimal k is ', optimal_k)

In [ ]:
#TABULATING ALL ACCURACY OF ALL ABOVE MODEL 
pred_values = pd.DataFrame({
    'Model': ['KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [score_knn, score_logreg, 
              score_ran, score_gaussian, score_perceptron, 
              score_sto, score_dec]})
pred_values.sort_values(by='Score', ascending=False)

In [ ]:
#EXPORTING THE PREDICTIONS OT CSV FILE
submission = pd.DataFrame({
        "id": test["id"],
        "target": y_pred
    })
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.describe()

In [ ]:
test.head(20)